In [1]:
import os
import sys
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
sys.path.append(os.path.abspath(os.path.join('..', 'src',)))

from models.classifiers.ridge import CustomRidge
from models.embeddings.openai_embedding import OpenAIEmbedding



In [2]:
embedding = OpenAIEmbedding(model_name='text-embedding-3-small')

AttributeError: 'OpenAIEmbedding' object has no attribute 'shape'

In [3]:
outputs_df = pd.read_parquet("../data/processed/outputs_openai_embeddings_v1.parquet")
ppas = outputs_df.PPAs_list.explode().drop_duplicates().sort_values().to_list()
# train_df = outputs_df[~outputs_df.test_set]
# test_df = outputs_df[outputs_df.test_set]
X_labels = outputs_df.columns[outputs_df.columns.str.contains("openai_embedding_small")]
# y_label = "PPAs_list"
oversampling = True

X_train = outputs_df[X_labels].to_numpy()
# y_train = train_df[y_label].to_numpy()
y_train = outputs_df[ppas].astype(int).to_numpy()

# X_test = test_df[X_labels].to_numpy()
# # y_test = test_df[y_label].to_numpy()
# y_test = test_df[ppas].astype(int).to_numpy()


ros = RandomOverSampler(random_state=42)
train_oversamples_df, _ = ros.fit_resample(outputs_df, y=outputs_df["primary_ppa"])
X_train_oversampled = train_oversamples_df[X_labels].to_numpy()
# y_train = train_df[y_label].to_numpy()
y_train_oversampled = train_oversamples_df[ppas].astype(int).to_numpy() 


In [10]:
outputs_df[ppas].sum().sort_values(ascending=False)

BL.2    16
BP.4    16
BP.1    13
BN.1    12
BL.4    11
BE.1    11
BL.3    10
BL.1     9
BL.6     9
BL.5     8
BN.2     8
BN.5     8
BP.3     8
BE.3     8
BP.2     7
BP.5     7
BE.4     6
BN.3     4
BN.4     4
BE.2     3
dtype: int64

In [10]:

ridge_clf = CustomRidge()
ridge_clf.fit(X_train_oversampled, y_train_oversampled)


CustomRidge()

In [27]:
import pickle

# Pickle the ridge_clf model
with open('../data/classifiers/ridge_clf.pkl', 'wb') as f:
    pickle.dump(ridge_clf, f)


In [28]:

# Unpickle the ridge clf model
with open('../data/classifiers/ridge_clf.pkl', 'rb') as f:
    ridge_clf_1 = pickle.load(f)

In [31]:
ridge_clf_1.classes_


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [38]:
ppas

['BE.1',
 'BE.2',
 'BE.3',
 'BE.4',
 'BL.1',
 'BL.2',
 'BL.3',
 'BL.4',
 'BL.5',
 'BL.6',
 'BN.1',
 'BN.2',
 'BN.3',
 'BN.4',
 'BN.5',
 'BP.1',
 'BP.2',
 'BP.3',
 'BP.4',
 'BP.5']

In [39]:
ppasappas

['BE.1',
 'BE.2',
 'BE.3',
 'BE.4',
 'BL.1',
 'BL.2',
 'BL.3',
 'BL.4',
 'BL.5',
 'BL.6',
 'BN.1',
 'BN.2',
 'BN.3',
 'BN.4',
 'BN.5',
 'BP.1',
 'BP.2',
 'BP.3',
 'BP.4',
 'BP.5']

In [36]:
e1 = embedding.generate("Under this Programme Priority Area (PPA), FAO will support national and local decision-makers to initiate, coordinate and scale up actions and investments towards urban and peri-urban food systems transformation to (i) reduce the level of urban poverty, food insecurity and malnutrition; (ii) address gender and social inequalities in cities and towns through improved urban and territorial food-policy, planning and integrated actions and inclusive governance; (iii) support an inclusive and sustainable rural transformation by leveraging urban demand for healthy diets and improved rural–urban and territorial linkages; (iv) promote sustainable management of natural resources (including natural ecosystems, green spaces, water, land, energy, and wild foods); (v) improve urban and territorial resilience of food systems to climatic and other shocks such as chronic stresses and extreme events; and (vi) strengthen partnerships at all levels and promote urban food systems alliances and coalitions with relevant actors.")

In [37]:
pd.DataFrame(ridge_clf.predict_proba(e1.reshape(1, -1)), columns=ppas).iloc[0].idxmax()

'BE.4'

'BL.4'